In [1]:
import numpy as np
import os
import pylab
import skimage
from skimage import io, filters, exposure, morphology, data, measure, segmentation, color
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.filters import sobel
from skimage.segmentation import active_contour
from skimage.measure import moments
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from scipy import ndimage as ndi
import math

In [2]:
INIT_DIR = os.getcwd()
IMAGE_DIR = INIT_DIR + '/GoodImages/'
os.chdir(IMAGE_DIR)
filelist = os.listdir()
os.chdir(INIT_DIR)
imagelist = []
for img in filelist:
    imagelist.append(IMAGE_DIR + img)

## Find Contours of Supplied Image

In [3]:
def bad_contours(contours):
    goodcon = np.argmax([len(c) for c in contours])
    return(contours[goodcon])

In [4]:
# Import image and plot grayscale histogram
def rockedges(image):
    # Read in image
    rock = io.imread(image)
    # convert to greyscale as our rocks have strange colors
    rockgray = color.rgb2gray(rock)
    # adjust the exposure as the lighting doesn't do a great job
    # of making the shadow/rock dilenation obvious
    rockgray = exposure.adjust_gamma(rockgray, gamma=.5, gain = 1)
    # smooth edges
    rockgray = filters.gaussian(rockgray, 6)
    # the numerical value (.2) here probably needs to change and is arbitrary.
    # I need to take new pictures as the lighting in the ones I have makes it hard to seperate 
    # shadow from painted rock color
    mask = rockgray > .2
    # prep to watershed segment the image
    elevationmap = sobel(rockgray)
    markers = np.zeros_like(rockgray)
    markers[rockgray < .2] = 1
    markers[rockgray > .2] = 2
    # watershed segmentation
    segmentation = morphology.watershed(elevationmap, markers)
    # find watershed contours
    cons = skimage.measure.find_contours(segmentation, 1)
    contour = bad_contours(cons)
    return contour, rockgray, rock

In [5]:
## find the center of mass
def center_mass(rockgray, contour):
    img = np.zeros_like(rockgray)
    x = contour[:,1]
    y = contour[:,0]
    rr, cc = skimage.draw.polygon(y,x)
    img[rr,cc] = 1
    m = moments(img)
    cc = m[0,1]/m[0,0]
    cr = m[1,0]/m[0,0]
    return cc, cr

In [6]:
def radius_angle(contour, cr, cc):
    radii = []
    angles = []
    y1 = contour[0][1]
    x1 = contour[0][0]
    for x, y in contour:
        radii.append(np.sqrt((cc - x)**2 + (cr - y)**2))
        angles.append(math.atan2((y1 - y), (x1 - x)))
    return radii, angles

In [7]:
def radius_plotter(rock, contour, cc, cr, rad, ang, name):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(rock)
    ax1.plot(contour[:, 1], contour[:, 0], linewidth=3, color = 'white')
    ax1.plot(cc, cr, color = 'white')
    ax2.scatter(ang, rad)
    ax2.set_ylabel('Radius (pixels)')
    ax2.set_xlabel('Angle (Radians)')
    plt.title(name)
    plt.savefig(name + '.pdf')
    plt.close()

In [8]:
def analyze_images(imagelist):
    n = 0
    INIT_DIR = os.getcwd()
    for img in imagelist:
        name = str(n)
        print('Working through image', name)
        contour, rockgray, rock = rockedges(img)
        cc, cr = center_mass(rockgray, contour)
        radii, angles = radius_angle(contour, cr, cc)
        radius_plotter(rock, contour, cc, cr, radii, angles, name)
        n = n + 1
    return

In [9]:
analyze_images(imagelist)

Working through image 0
Working through image 1
Working through image 2
Working through image 3
Working through image 4
Working through image 5
Working through image 6
Working through image 7
Working through image 8
Working through image 9
Working through image 10
Working through image 11
Working through image 12
Working through image 13
